# Explore the functions of the Quartz Solar Site API using Dev and Prod API

In [ ]:
# For loading credentials from .env file
import os
from dotenv import load_dotenv

# For data manipulation
import pandas as pd

# For API requests
import requests
import json

# For plotting
import plotly.express as px

In [1]:
environment = "prod"  # "dev" or "prod"

The access token can be inputed in two way:
1. Load the access token in from a .env file where the token is saved as a string variable valled "ACCESS_TOKEN"
    In this .env file have two access tokens called "PROD_ACCESS_TOKEN" and "DEV_ACCESS_TOKEN"
2. Making a request to the API using login crededntials to recieve the access token

In [ ]:
# A function to specify the environment and load the correct urls and credentials
def set_environment_variables(environment):
    load_dotenv()  # Ensure environment variables are loaded
    if environment == "prod":
        base_url = "https://api-site.quartz.solar"
        domain = "nowcasting-pro.eu.auth0.com"
        access_token = os.getenv('PROD_ACCESS_TOKEN')
        if access_token is None:
            raise ValueError("PROD_ACCESS_TOKEN not found in environment variables")
    elif environment == "dev":
        base_url = "http://uk-development-sites-api.eu-west-1.elasticbeanstalk.com"
        domain = "nowcasting-dev.eu.auth0.com"
        access_token = os.getenv('DEV_ACCESS_TOKEN')
        if access_token is None:
            raise ValueError("DEV_ACCESS_TOKEN not found in environment variables")
    else:
        raise ValueError("Invalid environment specified")
    return base_url, domain, access_token

base_url, domain, access_token = set_environment_variables(environment)

In [ ]:
# 2. Get access token from the API
username= "email" 
pwd = "password"

client_id = "QLMdXCCHMS9Zl5W2tDG3otpdcY7GeEnJ"
grant_type="password"

# Get access token
url = f"https://{domain}/oauth/token"
header = {'content-type': 'application/json'}
data = json.dumps({"client_id":client_id,"username":username,"password":pwd,"grant_type":grant_type,"audience":"https://api.nowcasting.io/"})
r = requests.post(url, data=data, headers=header)

# This notebook is setup to use method 1. If you want to use method 2, you can uncomment the line below
# access_token = r.json()['access_token']

In [ ]:
# Lets get the information about the sites available to us via the API using the access token
# and create a list of site UUIDs which we can use to pull the forecasts
url = base_url + "/sites"

print(url)

# Make the GET request to the API using the access token for authorization
r = requests.get(url=url, headers={"Authorization": "Bearer " + access_token})

# Check if the request was successful
if r.status_code == 200:
    data = r.json()
    sites_list = data['site_list']
    sites_info_df = pd.DataFrame(sites_list)
    print("Site data retrieved successfully:")

    # Create a sites list which we can use to pull the forecasts
    site_uuid_list = sites_info_df['site_uuid'].tolist()
else:
    print(f"Failed to retrieve site data. Status code: {r.status_code}")
    print("Response:", r.text)

sites_info_df

In [ ]:
# Check if the specific site_uuid exists in the site_uuid_list
site_uuid_to_check = "input_site_uuid_to_check"
if site_uuid_to_check in site_uuid_list:
    print(f"Site UUID {site_uuid_to_check} exists in the list.")
else:
    print(f"Site UUID {site_uuid_to_check} does not exist in the list.")

In [ ]:
# Lets get the forecast for a specific site using the site_uuid_list we created earlier
# or by inputing the string of the site_uuid directly
site_uuid  = site_uuid_list[1]
# site_uuid = "" # or input the site_uuid directly

print(f"Generating forecasts for site: {site_uuid}")

url = f"{base_url}/sites/{site_uuid}/pv_forecast"
r = requests.get(url=url, headers={"Authorization": "Bearer " + access_token})
    
if r.status_code == 200: # Check if the request was successful
    data = r.json()
    forecast_values = data['forecast_values']
    forecast_values_df = pd.DataFrame(forecast_values)
    forecast_creation_time = data['forecast_creation_datetime']
    print("PV forecast data for all sites retrieved successfully:")
else:
    print(f"Failed to retrieve PV forecast data for site {site_uuid}. Status code: {r.status_code}")
    print("Response:", r.text)

# Display the forecast creation time
print(f"Forecast creation time: {forecast_creation_time}")
print("Forecast Values: ")
print(forecast_values_df)

# Plotting the forecast values with site UUID in the title
title_with_uuid = f'PV Forecast Generation for Site {site_uuid}, forecast created at {forecast_creation_time} UTC'
fig = px.line(forecast_values_df, x='target_datetime_utc', y='expected_generation_kw', title=title_with_uuid)
fig.update_xaxes(title_text='Date and Time')
fig.update_yaxes(title_text='Expected Generation (kW)')
fig.show()

In [ ]:
def get_generation_data(site_uuid_list, access_token):
    all_forecast_data = []
    for site_uuid in site_uuid_list:
        url = f"{base_url}/sites/{site_uuid}/pv_forecast"
        r = requests.get(url=url, headers={"Authorization": "Bearer " + access_token})
        
        if r.status_code == 200:
            data = r.json()
            forecast_values = data['forecast_values']
            for forecast in forecast_values:
                forecast['site_uuid'] = site_uuid  # Add site_uuid to each forecast entry
            all_forecast_data.extend(forecast_values)
        else:
            print(f"Failed to retrieve PV forecast data for site {site_uuid}. Status code: {r.status_code}")
            print("Response:", r.text)
    df = pd.DataFrame(all_forecast_data)
    return df

# Get the first 20 sites from the list to pull forecasts for
site_uuid_list = sites_info_df['site_uuid'].tolist()[:20]
df_generation = get_generation_data(site_uuid_list, access_token)
print("PV forecast data for all sites retrieved successfully:")

df_generation

In [ ]:
# Creating a line chart for each site's generation data
fig = px.line(df_generation, x='target_datetime_utc', y='expected_generation_kw', color='site_uuid', 
              title=f'PV Forecast Generation for first 20 Sites, forecast created at {forecast_creation_time} UTC', labels='site_uuid')

# Updating axis titles
fig.update_xaxes(title_text='Date and Time')
fig.update_yaxes(title_text='Expected Generation (kW)')

# Updating legend font size to make it smaller
fig.update_layout(legend=dict(font=dict(size=7)))

# Showing the figure
fig.show()